# 🍎 Fruit Freshness: GAN vs VAE Karşılaştırması

Bu notebook, Fruit Freshness Classification veri seti üzerinde DCGAN ve VAE modellerinin karşılaştırmalı analizini içerir.

## İçerik
1. Kurulum ve Veri Hazırlığı
2. VAE Eğitimi
3. DCGAN Eğitimi
4. Model Karşılaştırması
5. Sonuçlar ve Analiz

## 1. Kurulum

In [ ]:
# Google Colab için kurulum
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Projeyi klonla (eğer GitHub'a yüklendiyse)
    # !git clone https://github.com/username/Derin_Final.git
    # %cd Derin_Final
    
    # Gerekli kütüphaneleri yükle
    !pip install kaggle -q
    
    # Kaggle API anahtarını ayarla
    # Kaggle'dan indirdiğiniz kaggle.json dosyasını yükleyin
    from google.colab import files
    print("Kaggle API token dosyasını yükleyin (kaggle.json):")
    # uploaded = files.upload()
    # !mkdir -p ~/.kaggle
    # !mv kaggle.json ~/.kaggle/
    # !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Gerekli kütüphaneleri import et
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm

# Proje modüllerini import et
import sys
sys.path.insert(0, '..')

from models.vae import VAE, vae_loss
from models.dcgan import Generator, Discriminator
from utils.dataloader import get_dataloader, denormalize
from utils.visualization import show_images, save_generated_images

# Device ayarı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

## 2. Veri Hazırlığı

In [ ]:
# Veri setini indir (ilk çalıştırmada)
# !kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification -p ../data --unzip

In [ ]:
# Hiperparametreler
BATCH_SIZE = 32
IMAGE_SIZE = 64
LATENT_DIM = 128  # VAE için
NOISE_DIM = 100   # GAN için
EPOCHS = 50
LR = 0.0002

In [ ]:
# Veri yükleyicileri oluştur
train_loader, val_loader = get_dataloader(
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    num_workers=2
)

# Örnek görüntüleri göster
sample_batch, _ = next(iter(train_loader))
show_images(sample_batch[:16], nrow=4, title="Örnek Eğitim Görüntüleri")

## 3. VAE Eğitimi

In [ ]:
# VAE modelini oluştur
vae = VAE(latent_dim=LATENT_DIM).to(device)
vae_optimizer = optim.Adam(vae.parameters(), lr=LR, betas=(0.5, 0.999))

# Parametre sayısı
vae_params = sum(p.numel() for p in vae.parameters())
print(f"VAE parameters: {vae_params:,}")

In [ ]:
# VAE eğitim döngüsü
vae_history = {'loss': [], 'recon': [], 'kl': []}
fixed_z_vae = torch.randn(16, LATENT_DIM, device=device)

print("VAE Eğitimi Başlıyor...")
for epoch in range(1, EPOCHS + 1):
    vae.train()
    epoch_loss, epoch_recon, epoch_kl = 0, 0, 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
    for images, _ in pbar:
        images = images.to(device)
        
        # Forward
        recon, mu, logvar = vae(images)
        loss, recon_loss, kl_loss = vae_loss(recon, images, mu, logvar)
        
        # Backward
        vae_optimizer.zero_grad()
        loss.backward()
        vae_optimizer.step()
        
        epoch_loss += loss.item()
        epoch_recon += recon_loss.item()
        epoch_kl += kl_loss.item()
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    # Epoch ortalamaları
    n = len(train_loader)
    vae_history['loss'].append(epoch_loss / n)
    vae_history['recon'].append(epoch_recon / n)
    vae_history['kl'].append(epoch_kl / n)
    
    # Her 10 epoch'ta örnek üret
    if epoch % 10 == 0 or epoch == 1:
        vae.eval()
        with torch.no_grad():
            samples = vae.decode(fixed_z_vae)
        show_images(samples, nrow=4, title=f"VAE - Epoch {epoch}")

print("VAE Eğitimi Tamamlandı!")

In [ ]:
# VAE loss grafikleri
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(vae_history['loss'])
axes[0].set_title('Total Loss')
axes[0].set_xlabel('Epoch')

axes[1].plot(vae_history['recon'])
axes[1].set_title('Reconstruction Loss')
axes[1].set_xlabel('Epoch')

axes[2].plot(vae_history['kl'])
axes[2].set_title('KL Divergence')
axes[2].set_xlabel('Epoch')

plt.tight_layout()
plt.show()

## 4. DCGAN Eğitimi

In [ ]:
# DCGAN modellerini oluştur
generator = Generator(noise_dim=NOISE_DIM).to(device)
discriminator = Discriminator().to(device)

criterion = nn.BCELoss()
g_optimizer = optim.Adam(generator.parameters(), lr=LR, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=LR, betas=(0.5, 0.999))

g_params = sum(p.numel() for p in generator.parameters())
d_params = sum(p.numel() for p in discriminator.parameters())
print(f"Generator parameters: {g_params:,}")
print(f"Discriminator parameters: {d_params:,}")

In [ ]:
# DCGAN eğitim döngüsü
gan_history = {'d_loss': [], 'g_loss': [], 'd_real': [], 'd_fake': []}
fixed_z_gan = torch.randn(16, NOISE_DIM, 1, 1, device=device)

print("DCGAN Eğitimi Başlıyor...")
for epoch in range(1, EPOCHS + 1):
    generator.train()
    discriminator.train()
    
    epoch_d_loss, epoch_g_loss = 0, 0
    epoch_d_real, epoch_d_fake = 0, 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
    for images, _ in pbar:
        batch_size = images.size(0)
        images = images.to(device)
        
        real_labels = torch.full((batch_size, 1, 1, 1), 0.9, device=device)
        fake_labels = torch.full((batch_size, 1, 1, 1), 0.0, device=device)
        
        # Train Discriminator
        d_optimizer.zero_grad()
        output_real = discriminator(images)
        d_loss_real = criterion(output_real, real_labels)
        
        noise = torch.randn(batch_size, NOISE_DIM, 1, 1, device=device)
        fake_images = generator(noise)
        output_fake = discriminator(fake_images.detach())
        d_loss_fake = criterion(output_fake, fake_labels)
        
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        d_optimizer.step()
        
        # Train Generator
        g_optimizer.zero_grad()
        noise = torch.randn(batch_size, NOISE_DIM, 1, 1, device=device)
        fake_images = generator(noise)
        output = discriminator(fake_images)
        g_loss = criterion(output, real_labels)
        g_loss.backward()
        g_optimizer.step()
        
        epoch_d_loss += d_loss.item()
        epoch_g_loss += g_loss.item()
        epoch_d_real += output_real.mean().item()
        epoch_d_fake += output.mean().item()
        
        pbar.set_postfix({'D': f'{d_loss.item():.4f}', 'G': f'{g_loss.item():.4f}'})
    
    # Epoch ortalamaları
    n = len(train_loader)
    gan_history['d_loss'].append(epoch_d_loss / n)
    gan_history['g_loss'].append(epoch_g_loss / n)
    gan_history['d_real'].append(epoch_d_real / n)
    gan_history['d_fake'].append(epoch_d_fake / n)
    
    # Her 10 epoch'ta örnek üret
    if epoch % 10 == 0 or epoch == 1:
        generator.eval()
        with torch.no_grad():
            samples = generator(fixed_z_gan)
        show_images(samples, nrow=4, title=f"DCGAN - Epoch {epoch}")

print("DCGAN Eğitimi Tamamlandı!")

In [ ]:
# GAN loss grafikleri
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(gan_history['d_loss'], label='Discriminator')
axes[0].plot(gan_history['g_loss'], label='Generator')
axes[0].set_title('GAN Losses')
axes[0].set_xlabel('Epoch')
axes[0].legend()

axes[1].plot(gan_history['d_real'], label='D(x)')
axes[1].plot(gan_history['d_fake'], label='D(G(z))')
axes[1].set_title('Discriminator Outputs')
axes[1].set_xlabel('Epoch')
axes[1].legend()

plt.tight_layout()
plt.show()

## 5. Model Karşılaştırması

In [ ]:
# Modelleri eval moduna al
vae.eval()
generator.eval()

# Örnek üret
with torch.no_grad():
    vae_samples = vae.sample(16, device)
    gan_samples = generator.generate(16, device)
    real_samples = next(iter(val_loader))[0][:16]

# Karşılaştırmalı görselleştirme
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, (images, title) in zip(axes, [
    (real_samples, "Gerçek Görüntüler"),
    (vae_samples, "VAE Üretimi"),
    (gan_samples, "DCGAN Üretimi")
]):
    if images.min() < 0:
        images = denormalize(images)
    grid = make_grid(images.cpu(), nrow=4, padding=2)
    ax.imshow(grid.permute(1, 2, 0).numpy())
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.axis('off')

plt.suptitle("Model Karşılaştırması", fontsize=18, fontweight='bold')
plt.tight_layout()
plt.savefig('../outputs/comparison/notebook_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Sonuç ve Analiz

### Görsel Kalite
- **VAE**: Üretilen görüntüler genellikle daha bulanık olur çünkü MSE loss kullanılır.
- **GAN**: Daha keskin ve detaylı görüntüler üretir, adversarial training sayesinde.

### Çeşitlilik
- **VAE**: Latent space'in tamamını kullandığı için çeşitlilik genellikle daha yüksek.
- **GAN**: Mode collapse riski vardır, bazı modlar öğrenilmeyebilir.

### Eğitim Kararlılığı
- **VAE**: Kararlı eğitim, loss sürekli azalır.
- **GAN**: Generator ve Discriminator arasında denge gerektirir, dengesiz olabilir.

In [ ]:
# Modelleri kaydet
torch.save(vae.state_dict(), '../checkpoints/vae_notebook.pt')
torch.save(generator.state_dict(), '../checkpoints/generator_notebook.pt')
print("Modeller kaydedildi!")